In [67]:
from federated import agglomerate
import torch
from time import sleep
from server import move_left, move_forward, move_right, stop_robot
from server import display_camera_stream
from server import get_state, set_drive, get_weights, set_weights
import numpy as np
import random

In [108]:
def x_y_to_action(x=float, y=float):
    """
    X and Y are clamped to between 0 and 1, -1, -1 if not applicable
    """
    if x < 0 or y < 0:
        return (0, 1)

    if x < 0.5:
        direction = 1
    else:
        direction = -1
    turn_speed = abs(x-0.5)/0.5

    forward_speed = abs(y-0.5)/0.5

    return (forward_speed*0, turn_speed*direction)

In [109]:
def PSO(colour_cords: list[dict], colors):
    best_score = (-1, -1)
    best_robot = 0
    acts = [(0,0)]*len(colour_cords)
    sees_red = [False]*len(colour_cords)
    desired_coords_per_robot = [[]] * len(colour_cords)

    # Check if any robot sees red
    for index, robot in enumerate(colour_cords):
        if tuple(robot["red"]) > (0, 0):
            # Red is a desirable colour
            desired_coords_per_robot[index].append(robot["red"])
            sees_red[index] = True

    for index, robot in enumerate(colour_cords):
        for colour_i, colour in enumerate(colors):
            if colour_i == 0:
                continue
            if tuple(robot[colour]) > (0,0):
                # Add one to offset from the red box
                robot_color_index = colour_i - 1
                if sees_red[robot_color_index]:
                    # A robot that sees red is a desirable robot
                    desired_coords_per_robot[index].append(robot[colour])
                    
    for index, robot in enumerate(colour_cords):
        if len(desired_coords_per_robot[index]) > 0:
            x_vals, y_vals = zip(*desired_coords_per_robot[index])

            # Get random point within bounds
            x = random.uniform(min(x_vals), max(x_vals))
            y = random.uniform(min(y_vals), max(y_vals))

            action = x_y_to_action(x, y)
        else:
            action = (0, 0.5)  # Default action if no desirable coordinates
        acts[index] = action

    """
    for index, robot in enumerate(colour_cords): 
        box_cords = robot["red"]
        if box_cords[0]>= 0 and box_cords[1]>= 0:
            score = tuple(np.subtract((0.5, 0.5), tuple(box_cords)))
        else:
            score = (-1, -1)
        acts[index] = x_y_to_action(*box_cords)
        
        if score > tuple(best_score):
            best_robot = index
            best_score = score
    
    if len(colour_cords) > 1 and best_score > (0, 0):
        for index, robot in enumerate(colour_cords):
            if index == best_robot:
                continue
            best_robot_colour = colors[best_robot]
            robot_cords = robot[best_robot_colour]
            if robot_cords[0]>= 0 and robot_cords[1]>= 0:
                random_int = np.random.random(1)[0]
                acts[index] = random_int * acts[index] + (1-random_int) * x_y_to_action(*robot[best_robot_colour]) 
    """

    return acts

In [110]:
# Test connection
IPS = ["194.47.156.221", "194.47.156.22"]
COLOURS = ["pink", "yellow"]
for i in range(0, 100):
    states = []
    for index, IP in enumerate(IPS):
        states.append(get_state(IP))
states

[{'red': [0.6071428571428571, 0.35714285714285715],
  'orange': [-1, -1],
  'yellow': [-1, -1],
  'blue': [-1, -1],
  'pink': [-1, -1]},
 {'red': [-1, -1],
  'orange': [-1, -1],
  'yellow': [-1, -1],
  'blue': [-1, -1],
  'pink': [-1, -1]}]

In [111]:
USE_FL = False
weights = [None]*len(IPS)
states = [(0,0)]*len(IPS)
for i in range(0, 20):
    states = []
    for index, IP in enumerate(IPS):
        state = get_state(IP)
        print(f"{IP=} | {COLOURS[index]} | {state=}")
        states.append(state)

    descisions = PSO(states, COLOURS)

    for index, IP in enumerate(IPS):
        forward = np.clip(descisions[index], a_min=0.1, a_max=0.3)[0]
        turn = np.clip(descisions[index], a_min=-0.5, a_max=0.5)[1]
        move_robot =  set_drive(IP, forward, turn)

    sleep(1) 

    for index, IP in enumerate(IPS):
        stop_robot(IP)
    
    if USE_FL and i % 10 == 0:
        for index, IP in enumerate(IPS):
            weights[index] = get_weights(IP)

        agg_weight = agglomerate(weights)
        
        for index, IP in enumerate(IPS):
            send_weights = set_weights(IP, agg_weight)
            pass

for index, IP in enumerate(IPS):
    stop_robot(IP)
if USE_FL:
    for index, IP in enumerate(IPS):
        weights[index] = get_weights(IP)

    agg_weight = agglomerate(weights)
    torch.save(agg_weight, "model.pth")

IP='194.47.156.221' | pink | state={'red': [0.5803571428571429, 0.33035714285714285], 'orange': [-1, -1], 'yellow': [-1, -1], 'blue': [-1, -1], 'pink': [0.8571428571428571, 0.4732142857142857]}
IP='194.47.156.22' | yellow | state={'red': [-1, -1], 'orange': [-1, -1], 'yellow': [-1, -1], 'blue': [-1, -1], 'pink': [-1, -1]}
Drive set successfully
Drive set successfully
Stop command executed successfully
Stop command executed successfully
IP='194.47.156.221' | pink | state={'red': [0.5044642857142857, 0.35267857142857145], 'orange': [-1, -1], 'yellow': [-1, -1], 'blue': [-1, -1], 'pink': [0.8616071428571429, 0.6339285714285714]}
IP='194.47.156.22' | yellow | state={'red': [-1, -1], 'orange': [-1, -1], 'yellow': [-1, -1], 'blue': [-1, -1], 'pink': [-1, -1]}
Drive set successfully
Drive set successfully
Stop command executed successfully
Stop command executed successfully
IP='194.47.156.221' | pink | state={'red': [0.4375, 0.375], 'orange': [-1, -1], 'yellow': [-1, -1], 'blue': [-1, -1], 'p

In [ ]:
states

In [ ]:
for index, IP in enumerate(IPS):
    stop_robot(IP)